# COMP5329 - Deep Learning
## Tutorial 2 - Multilayer Neural Network

**Semester 1, 2022**

**Objectives:**

* To understand the multi-layer perceptron.
* To become familiar with backpropagation.



## Loading the packages

In [1]:
import numpy as np
import matplotlib.pyplot as pl
from ipywidgets import interact, widgets
from matplotlib import animation

## Import Data 

In [2]:
train_data = np.load('train_data.npy')
tr_label = np.load('train_label.npy')
test_data = np.load('test_data.npy')
ts_label = np.load('test_label.npy')

print("shape of tran_data is {} and of tran_label is {}".format(train_data.shape, tr_label.shape))
print("shape of test_data is {} and of test_label is {}".format(test_data.shape, ts_label.shape))

data_stats = {
    'Minimum': np.min(train_data),
    'Maximum': np.max(train_data),
    'Mean': np.mean(train_data),
    'Standard Deviation': np.std(train_data),
    "label example" : tr_label[:10]
}

data_stats

shape of tran_data is (50000, 128) and of tran_label is (50000, 1)
shape of test_data is (10000, 128) and of test_label is (10000, 1)


{'Minimum': -23.41520966868848,
 'Maximum': 25.581360537720347,
 'Mean': -1.593740250882547e-14,
 'Standard Deviation': 1.1688636581970187,
 'label example': array([[6],
        [9],
        [9],
        [4],
        [1],
        [1],
        [2],
        [7],
        [8],
        [3]], dtype=uint8)}

## Pre-processing

Obviously, the mean and deviation of training datasets are really close to 0 and 1 respectively, which means that the data can be used directly as a normalized data. However this time I still choose to process normalization for precision.

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train_data_standardized = scaler.fit_transform(train_data)
test

standardized_mean = np.mean(train_data_standardized)
standardized_std = np.std(train_data_standardized)

standardized_mean, standardized_std


(-1.4710455076283324e-19, 1.0000000000000002)

In [4]:
def convert_to_one_hot(labels, num_classes):
    # Create an array of zeros with shape (num_samples, num_classes)
    one_hot_labels = np.zeros((labels.size, num_classes))
    
    # Use numpy advanced indexing to set the appropriate elements to one
    one_hot_labels[np.arange(labels.size), labels.flatten()] = 1
    
    return one_hot_labels

train_label = convert_to_one_hot(tr_label, 10)
test_label = convert_to_one_hot(ts_label, 10)

train_label

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

## Definition of some activation functions

Linear
$$output = x$$

Tanh  
$$output = tanh(x)$$  

Sigmoid
$$output = \frac {1}{1 + e^{-x}}$$

In [5]:
# create a activation class
# for each time, we can initiale a activation function object with one specific function
# for example: f = Activation("tanh")  means we create a tanh activation function.
# you can define more activation functions by yourself, such as relu!

class Activation(object):
    def __tanh(self, x):
        return np.tanh(x)
    def __tanh_deriv(self, a):
        # a = np.tanh(x)
        return 1.0 - a**2
    
    def __logistic(self, x):
        return 1.0 / (1.0 + np.exp(-x))
    def __logistic_deriv(self, a):
        # a = logistic(x)
        return  a * (1 - a )
    
    def __ReLU(self, x):
        return np.maximum(0,x)
    def __ReLU_deriv(self, a):
        #a = ReLU(x)
        return np.where(a > 0, 1, 0)
    
    def __softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)
    def __softmax_deriv(self, x):
       
        pass

    def __init__(self,activation='tanh'):
        if activation == 'logistic':
            self.f = self.__logistic
            self.f_deriv = self.__logistic_deriv
        elif activation == 'tanh':
            self.f = self.__tanh
            self.f_deriv = self.__tanh_deriv
        elif activation == 'ReLU':
            self.f = self.__ReLU
            self.f_deriv = self.__ReLU_deriv
        elif activation == 'softmax':
            self.f = self.__softmax
            self.f_deriv = self.__softmax_deriv


### Define HiddenLayer

$$output = f\_act(\sum_{i=0}^{1}{(I_{i} * W_{i})} + b)$$

In [6]:
# now we define the hidden layer for the mlp
# for example, h1 = HiddenLayer(10, 5, activation="tanh") means we create a layer with 10 dimension input and 5 dimension output, and using tanh activation function.
# notes: make sure the input size of hiddle layer should be matched with the output size of the previous layer!

class HiddenLayer(object):
    def __init__(self,n_in, n_out,
                 activation_last_layer='softmax',activation='ReLU', W=None, b=None):
        """
        Typical hidden layer of a MLP: units are fully-connected and have
        sigmoidal activation function. Weight matrix W is of shape (n_in,n_out)
        and the bias vector b is of shape (n_out,).

        NOTE : The nonlinearity used here is tanh

        Hidden unit activation is given by: tanh(dot(input,W) + b)

        :type n_in: int
        :param n_in: dimensionality of input

        :type n_out: int
        :param n_out: number of hidden units

        :type activation: string
        :param activation: Non linearity to be applied in the hidden
                           layer
        """
        self.input=None
        self.activation=Activation(activation).f 

        # activation deriv of last layer
        self.activation_deriv=None
        if activation_last_layer:
            self.activation_deriv=Activation(activation_last_layer).f_deriv

        # we randomly assign small values for the weights as the initiallization
        # self.W = np.random.uniform(
        #         low=-np.sqrt(6. / (n_in + n_out)),
        #         high=np.sqrt(6. / (n_in + n_out)),
        #         size=(n_in, n_out)
        # )
        # if activation == 'logistic':
        #     self.W *= 4
            
        # if activation == 'relu':
        # apply HE initialization
        self.W = np.random.randn(n_in, n_out) * np.sqrt(2. /n_in)

        # we set the size of bias as the size of output dimension
        self.b = np.zeros(n_out,)

        # we set he size of weight gradation as the size of weight
        self.grad_W = np.zeros(self.W.shape)
        self.grad_b = np.zeros(self.b.shape)


    # the forward and backward progress (in the hidden layer level) for each training epoch
    # please learn the week2 lec contents carefully to understand these codes.
    def forward(self, input):
        '''
        :type input: numpy.array
        :param input: a symbolic tensor of shape (n_in,)
        '''
        lin_output = np.dot(input, self.W) + self.b
        self.output = (
            lin_output if self.activation is None
            else self.activation(lin_output)
        )
        self.input=input
        return self.output

    def backward(self, delta, output_layer=False):

        self.grad_W = np.atleast_2d(self.input).T.dot(np.atleast_2d(delta))
        self.grad_b = delta
        if self.activation_deriv is not None:
            delta = delta.dot(self.W.T) * self.activation_deriv(self.input)
        else:
            delta = delta.dot(self.W.T)
        return delta




## The MLP

The class implements a MLP with a fully configurable number of layers and neurons. It adapts its weights using the backpropagation algorithm in an online manner.

In [7]:
class MLP:
    """
    """

    # for initiallization, the code will create all layers automatically based on the provided parameters.
    def __init__(self, layers, activation=[None,'softmax','softmax']):
        """
        :param layers: A list containing the number of units in each layer.
        Should be at least two values
        :param activation: The activation function to be used. Can be
        "logistic" or "tanh"
        """
        ### initialize layers
        self.layers=[]
        self.params=[]

        self.activation=activation
        for i in range(len(layers)-1):
            self.layers.append(HiddenLayer(layers[i],layers[i+1],activation[i],activation[i+1]))

    # forward progress: pass the information through the layers and out the results of final output layer
    def forward(self,input):
        for layer in self.layers:
            output=layer.forward(input)
            input=output
        return output

    # define the objection/loss function, we use mean sqaure error (MSE) as the loss
    # you can try other loss, such as cross entropy.
    # when you try to change the loss, you should also consider the backward formula for the new loss as well!
    def criterion_MSE(self,y,y_hat):
        activation_deriv=Activation(self.activation[-1]).f_deriv
        # MSE
        error = y-y_hat
        loss=error**2
        # calculate the MSE's delta of the output layer
        delta=-2*error*activation_deriv(y_hat)
        # return loss and delta
        return loss,delta
    
    def Cross_Entropy(self,y,y_hat, epsilon = 1e-12):
        # Ensure the predictions are within the range (epsilon, 1-epsilon) for numerical stability
        y_hat = np.clip(y_hat, epsilon, 1. - epsilon)
        # Compute cross-entropy loss for each class and sum
        cross_entropy = -np.sum(y * np.log(y_hat))
        # Calculate the gradient for the output layer
        delta = y_hat - y
        return cross_entropy, delta
                        
    # def update_adam(self,X,y,learning_rate = 0.1, epochs = 100, epsilon = 1e-12):
    #     for layer in self.layers:
    #         layer.W = beta1

            
    # backward progress
    def backward(self,delta):
        delta=self.layers[-1].backward(delta,output_layer=True)
        for layer in reversed(self.layers[:-1]):
            delta=layer.backward(delta)

    # update the network weights after backward.
    # make sure you run the backward function before the update function!
    

    

    def update(self,lr):
        for layer in self.layers:
            layer.W -= lr * layer.grad_W
            layer.b -= lr * layer.grad_b

    # define the training function
    # it will return all losses within the whole training process.
    def fit(self,X,y,learning_rate=0.1, epochs=100):
        """
        Online learning.
        :param X: Input data or features
        :param y: Input targets
        :param learning_rate: parameters defining the speed of learning
        :param epochs: number of times the dataset is presented to the network for learning
        """
        X=np.array(X)
        y=np.array(y)
        to_return = np.zeros(epochs)

        for k in range(epochs):
            loss=np.zeros(X.shape[0])
            for it in range(X.shape[0]):
                i=np.random.randint(X.shape[0])

                # forward pass
                y_hat = self.forward(X[i])

                # backward pass
                # loss[it],delta=self.criterion_MSE(y[i],y_hat)
                loss[it],delta=self.Cross_Entropy(y[i],y_hat)
                self.backward(delta)
                y
                # update
                self.update(learning_rate)
            to_return[k] = np.mean(loss)
        return to_return

    # define the prediction function
    # we can use predict function to predict the results of new data, by using the well-trained network.
    def predict(self, x):
        x = np.array(x)
        output = np.zeros(x.shape[0])
        for i in np.arange(x.shape[0]):
            output[i] = self.forward(x[i,:])
        return output

## Learning

In [8]:
# ### Try different MLP models
# # we can compare the loss change graph to under how the network parameters (such as number of layers and activation functions),
# # could affect the performance of network.
# nn = MLP([2,4,8,1], [None,'ReLU','ReLU','ReLU'])
# input_data = train_data[:,0:2]
# output_data = train_data[:,2]
# MSE = nn.fit(input_data, output_data, learning_rate=0.001, epochs=50)
# print('loss:%f'%MSE[-1])
# pl.figure(figsize=(15,4))
# pl.plot(MSE)
# pl.grid()

In [12]:
### Try different MLP models
# we can compare the loss change graph to under how the network parameters (such as number of layers and activation functions),
# could affect the performance of network.
nn = MLP([2,4,4,10], [None,'ReLU','ReLU','softmax'])
input_data = train_data[:,0:2]
output_data = train_data[:,2]
CE = nn.fit(input_data, output_data, learning_rate=0.1, epochs=50)
print('loss:%f'%CE[-1])
pl.figure(figsize=(15,4))
pl.plot(CE)
pl.grid()

## Testing

In [ ]:
output = nn.predict(input_data)

ValueError: setting an array element with a sequence.

In [ ]:
# visualizing the predict results
# notes: since we use tanh function for the final layer, that means the output will be in range of [0,1]
pl.figure(figsize=(8,6))
pl.scatter(output_data, output, s=100)
pl.xlabel('Targets')
pl.ylabel('MLP output')
pl.grid()

In [ ]:
# create a mesh to plot in
xx, yy = np.meshgrid(np.arange(-2, 2, .02),np.arange(-2, 2, .02))

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, m_max]x[y_min, y_max].
Z = nn.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)

pl.figure(figsize=(15,7))
pl.subplot(1,2,1)
pl.pcolormesh(xx, yy, Z>0, cmap='cool')
pl.scatter(input_data[:,0], input_data[:,1], c=[(['b', 'r'])[d>0] for d in output_data], s=100)
pl.xlim(-2, 2)
pl.ylim(-2, 2)
pl.grid()
pl.title('Targets')
pl.subplot(1,2,2)
pl.pcolormesh(xx, yy, Z>0, cmap='cool')
pl.scatter(input_data[:,0], input_data[:,1], c=[(['b', 'r'])[d>0] for d in output], s=100)
pl.xlim(-2, 2)
pl.ylim(-2, 2)
pl.grid()
pl.title('MLP output')

### the figure on the left shows the ground true label of each data
### the figure on the eright shows the predict label of each data with MLP model.
### Based on the visualization result, we can find that network learned a boundary between positive and negative data!
